# Draft: Pandeia Tutorial

***

## Introduction
**Pandeia** is a high-fidelity exposure time calculator developed by the Space Telescope Science Institute to provide optimal observing setups for user-created scenes. It supports the Roman Wide Field Imager (WFI) as well as the James Webb Space Telescope's full complement of instruments.

Due to the complexity of its simulations, Pandeia is best used on scenes that encompass 5% of a single WFI detector. (To simulate larger areas, see the STIPS Tutorial.) The parameters returned can then help judge the feasibility of and best settings for a fully fledged observing proposal for Roman.

This notebook walks through brief examples of common use cases. Before you begin, be sure that your environment is properly equipped for Pandeia – that includes both installation and downloading the required data files.

## Imports

Besides the Pandeia-related imports, `scipy.optimize.minimize_scalar` will help with optimizing signal-to-noise ratios in upcoming examples.

In [ ]:
from pandeia.engine.calc_utils import build_default_calc
from pandeia.engine.perform_calculation import perform_calculation
from scipy.optimize import minimize_scalar

We set `FILTER` as global variable before beginning since all examples make use of the same F129 imaging filter.

In [ ]:
FILTER = 'f129'

***

## Examples

### Calculate a scene's signal-to-noise ratio

Simulate a scene with the following observing setup:
* Multi-accumulation (MA) table: "High Latitude Wide Area Survey – Imaging" (default)
* Filter: F129 (defined earlier)
* Number of exposures: 3

In [ ]:
calc = build_default_calc('roman', 'wfi', 'imaging')

calc['configuration']['instrument']['filter'] = FILTER
calc['configuration']['detector']['nexp'] = 3

Next, change the default point source flux to 25 AB magnitudes.

In [ ]:
mag = 25
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'

Finally, perform the signal to noise calculation and print the result.

In [ ]:
report = perform_calculation(calc)
sn = report['scalar']['sn']
print(f'Estimated S/N: {sn:.2f}')

_This step may generate a WARNING from synphot that the spectrum is extrapolated, which can be ignored._

_Running Pandeia for Roman may return a warning such as: `if np.log(abs(val)) < -1*precision and val != 0.0`. This is related to a JWST-specific test for float precision, and can be ignored in this case._

### Estimate optimal limiting magnitude

Given an exposure configuration and a set signal-to-noise ratio, you can determine an optimal limiting magnitude by using Pandeia to simulate a range of scenes at different magnitudes. The following functions show how to achieve this result.

In [ ]:
def _mag2sn_(mag, calc, sntarget):
    """
    Optimize a S/N ratio given a magnitude. This is a helper function used as
    an argument for scipy's minimize_scalar() as used in compute_mag().
    
    Parameters
    ----------
    mag: TYPE??
        The magnitude used in an iteration of minimize_scalar()
    calc: TYPE??
        A Pandeia calculation object
    sntarget: TYPE??
        Required S/N from the matching argument of compute_mag()
    
    """
    calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    etc = perform_calculation(calc)['scalar']
    
    return (sntarget - etc['sn'])**2


def compute_mag(filt, sn, nexp, bracket=(18., 30.), xtol=1e-4):
    """
    Method to compute the magnitude from S/N and number of exposures
 
    Parameters
    ----------
    filt : str
        Name of Roman WFI filter
    sn : float
        Required S/N
    nexp : int
        Number of exposures
    bracket : tuple
        Range of magnitudes to test. default: (18., 30.)
    xtold: float, default 1e-4
        Target tolerance for minimizer
 
    Returns
    -------
    mag : float
        Optimal magnitude for specified S/N and number of exposures
    report: dict
        Pandeia dictionary with optimal parameters
    """
 
    # Set up default Roman observation
    calc = build_default_calc('roman', 'wfi', 'imaging')
 
    # Modify defaults to place a source with an AB magnitude
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
    calc['scene'][0]['spectrum']['normalization']['norm_waveunit'] = 'um'
      
    # Set number of exposures and filter
    calc['configuration']['detector']['nexp'] = nexp
    calc['configuration']['instrument']['filter'] = filt
 
    res = minimize_scalar(_mag2sn_, bracket=bracket, args=(calc, sn),
                          method='brent', options={'xtol': xtol})
    mag = res['x']
    calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    report = perform_calculation(calc)
    
    return mag, report

Calculate the optimal limiting magnitude given a **signal-to-noise ratio of 5** and the following observing setup:
* Number of exposures: 10
* Filter: F129 (defined earlier)

In [ ]:
sn = 5
nexp = 10

mag, report = compute_mag(FILTER, sn, nexp)
print(f'Estimated magnitude: {mag:.2f}') # OJO: EQUALS 27.68?

### Determine optimal number of exposures

If you instead provide a desired magnitude limit and a set signal-to-noise ratio, you can calculate the optimal number of exposures needed to achieve such results for a given MA table. As such, we create alternatives to last example's functions that target a number of exposures instead of a magnitude value.

In [ ]:
def _nexp2sn_(nexp, calc, sntarget):
    """
    Optimize a S/N ratio given a number of exposures. This is a helper function
    used as an argument for scipy's minimize_scalar() as used in compute_mag().
    
    Parameters
    ----------
    nexp : int
        The number of exposures used in an iteration of minimize_scalar()
    calc : 
        A Pandeia calculation object
    sntarget : 
        Required S/N from the matching argument of compute_mag()
    """
    calc['configuration']['detector']['nexp'] = int(nexp)
    etc = perform_calculation(calc)['scalar']
    
    return (sntarget - etc['sn'])**2


def compute_nexp(filt, sn, mag, bracket=(1, 1000), xtol=0.1):
    """
    Method to compute the number of exposures from S/N and magnitude
 
    Parameters
    ----------
    filt : str
        Name of Roman WFI filter
    sn : float
        Required S/N
    mag : float
        AB Magnitude of source
    bracket : tuple, default (1, 1000)
        Range of magnitudes to test
    xtold: float, default 0.1
        Target tolerance for minimizer
 
    Returns
    -------
    nexp : float
        Optimal number of exposures for specified S/N and magnitude
    report: dict
        Pandeia dictionary with optimal parameters
    exptime: float
        Exposure time for optimal observation
    """
 
    # Set up default Roman observation
    calc = build_default_calc('roman', 'wfi', 'imaging')
 
    # Modify defaults to place a source with an AB magnitude
    calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
    calc['scene'][0]['spectrum']['normalization']['norm_waveunit'] = 'um'
      
    # Set filter
    calc['configuration']['instrument']['filter'] = filt
 
    # Check that the minimum of 1 exposure has a S/N lower than requested,
    # otherwise there is no sense in attempting to minimize nexp.
    calc['configuration']['detector']['nexp'] = 1
    report = perform_calculation(calc)
    
    if report['scalar']['sn'] > sn:
        nexp = 1
    else:
        res = minimize_scalar(_nexp2sn_, bracket=bracket, bounds=bracket,
                              args=(calc, sn), method='bounded',
                              options={'xatol': xtol})
        nexp = int(res['x'])
        calc['configuration']['detector']['nexp'] = nexp
        report = perform_calculation(calc)
 
        # this generally returns a S/N less than the required amount.
        # let's ensure that we get *AT LEAST* the required S/N for 2 reasons:
        # 1) better to err on the side of caution
        # 2) make code consistent with the above if clause
        if report['scalar']['sn'] < sn:
            nexp += 1
             
    exptime = report['scalar']['total_exposure_time']
         
    return nexp, report, exptime

Given the following observing setup:
* Multi-accumulation (MA) table: "High Latitude Wide Area Survey – Imaging" (default)
* Filter: F129 (defined earlier)

...and these observation parameters:
* Signal-to-noise ratio: 20
* Source magnitude: 26

...we can use the functions above to determine an optimal number of exposures. For completeness, we also print the actual signal-to-noise ratio achieved and the exposure time needed for that result.

In [ ]:
sn = 20.
mag = 26.
 
nexp, etc, exptime = compute_nexp(FILTER, sn, mag)
print(f'number of exposures: {nexp}')
print(f'actual S/N reached: {etc["scalar"]["sn"]:.2f}')
print(f'Exposure time: {exptime:.2f}')

### Modify spectral energy distribution

While previous examples assume a flat spectral energy distribution (SED), Pandeia also offers the ability to use a Phoenix model for more complex cases. This example uses the following observing setup:

* Multi-accumulation (MA) table: "High Latitude Wide Area Survey – Imaging" (default)
* Filter: F129 (defined earlier)
* Number of exposures: 3

In [ ]:
calc = build_default_calc('roman', 'wfi', 'imaging')

nexp = 3
calc['configuration']['detector']['nexp'] = nexp
calc['configuration']['instrument']['filter'] = FILTER

Next, calculate a signal-to-noise ratio after making the following modifications to the spectra:
* Magnitude: 25 AB magnitudes
* SED type: A0V (from Phoenix models)

In [ ]:
mag = 25
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
 
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'phoenix'
calc['scene'][0]['spectrum']['sed']['key'] = 'a0v'

In [ ]:
report = perform_calculation(calc)
sn = report['scalar']['sn']
print(f'Estimated S/N: {sn:.2f}')

## Aditional Resources

- The Roman User Documentation's ["Pandeia for Roman"](https://roman-docs.stsci.edu/simulation-tools-handbook-home/pandeia-for-roman) page and associated overview.
- Full API references for [Pandeia Engine inputs](https://outerspace.stsci.edu/display/PEN/Pandeia+Engine+Input+API) and [Pandeia Engine outputs](https://outerspace.stsci.edu/display/PEN/Pandeia+Engine+Output+API).
- The [Roman Help Desk](https://roman-docs.stsci.edu/roman-help-desk-at-stsci), an official outlet for user questions about Pandeia.

## About this notebook

**Author:** Justin Otor, Staff Scientist II.  
**Updated In:** 2024-05

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 